In [ ]:
# @title Подключение к Гугл Диску
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Подключение whisper и youtube-dl
!pip install git+https://github.com/ytdl-org/youtube-dl.git
!pip install git+https://github.com/openai/whisper.git


In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 faiss-cpu==1.7.4
!pip install -qq python-docx
!pip install -U pytube
from IPython.display import clear_output
# clear_output()

In [ ]:
# @title Устанавливаем необходимые библиотеки
import whisper
import os
import gdown
import requests
import re
import time
from IPython.display import HTML, clear_output
import subprocess
from pathlib import Path
import json
from pytube import YouTube
import tiktoken
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import ipywidgets as widgets
from IPython.display import display
from tqdm.auto import tqdm
import getpass
import pickle
from urllib.request import urlopen
import openai
import subprocess
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import urllib

In [ ]:
# @title Здесь нужно вставить ссылку на видео на иностранном языке
# указываем ссылку на нужное нам видео на ютуб
yt_urls = ['https://www.youtube.com/watch?v=880TBXMuzmk&pp=ygUhR29vZ2xlIERldmVsb3BlcnMgWW91VHViZSBDaGFubmVs']
YouTube_video_title = "Разработчики Google о будущем искусственного интеллекта"

## Получение аудио-дорожки из видео по ссылке и загрузка в колаб

In [ ]:
def my_mkdirs(folder):
  if os.path.exists(folder)==False:
    os.makedirs(folder)
my_mkdirs('/content/')
output_folder = '/content/drive/MyDrive/data_structure/'
my_mkdirs(output_folder)

In [ ]:
# Получаем первую (и единственную) ссылку из списка yt_urls
url = yt_urls[0]

# Используем youtube-dl для получения имени файла, который будет сохранен
file_name = !youtube-dl $url -f 'bestaudio[ext=m4a]' --get-filename -o 'tmp/%(title)s.m4a'

# Загружаем аудио с лучшим качеством (в формате m4a)
!youtube-dl $url -f 'bestaudio[ext=m4a]' -o 'tmp/%(title)s.m4a'


In [ ]:
!ls "/content/"

In [ ]:
# Используем Whisper для обработки аудиофайла
# УКАЗЫВАЕМ СВОЕ НАЗВАНИЕ ФАЙЛА .mp4!
%%time
!whisper "/content/The AI revolution - Google's developers on the future of artificial intelligence _ 60 Minutes-880TBXMuzmk.mp4" --model large

In [ ]:
!ls '/content/drive/MyDrive/data_structure/'

In [ ]:
import glob
import shutil

source_directory = '/content/'
destination_directory = '/content/drive/MyDrive/data_structure/'

# Находим первый файл .txt в папке /content/
txt_files = glob.glob(os.path.join(source_directory, '*.txt'))

if txt_files:
    # Берем только первый файл
    file = txt_files[0]
    destination_path = os.path.join(destination_directory, os.path.basename(file))

    # Перемещаем файл
    shutil.move(file, destination_path)
    print(f'Файл {file} перемещен в {destination_path}')
else:
    print('Файлы .txt не найдены в директории /content/')

Выводим текст, который получился

In [ ]:
# Путь к файлу на Google Драйве

file_path = "/content/drive/MyDrive/data_structure/The AI revolution - Google's developers on the future of artificial intelligence _ 60 Minutes-880TBXMuzmk.txt"

# Чтение и вывод содержимого файла
try:
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print(f'Файл не найден: {file_path}')

Итак, мы получили транскрибацию видео по ссылке с ютуб.

In [ ]:
# По запросу вводим ключ OPENAI_API_KEY и не забываем нажать ENTER.

import getpass
import os
import openai
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [ ]:
# @title Установка и импорты
import shutil
from langchain.docstore.document import Document
from langchain.llms import OpenAI

#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
# Разбивка на главы, аннотация, краткие выводы, критика материала, вопросы по докладу, ...

In [ ]:

chunk_size = 8000
temperature = 0

system = '''Ты гений текста, копирайтинга, писательства, отлично разбирающийся в вопросах искусственного интеллекта. Твоя задача, используя последовательные
чанки исходного текста, по всем правилам написать научный студенческий доклад на русском языке.
'''
user = """Пожалуйста, шаг за шагом, объедини все чанки исходного текста в один доклад. Доклад должен содержать только научную и техническую информацию.
Убери из доклада всю 'воду', безжалостно удаляй второстепенные пассажи, оставляй только самую суть. Ничего не придумывай от себя."""


In [ ]:
# @title Функции
# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить переполнение текста за границы ячейки вывода и обеспечить его перенос.
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-3.5-turbo-0301'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')

    if model in ['gpt-3.5-turbo-0301', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo']:
        num_tokens = 0

        for message in messages:
            num_tokens += 4

            for key, value in message.items():
                num_tokens += len(encoding.encode(value))

                if key == 'name':
                    num_tokens -= 1

        num_tokens += 2
        return num_tokens

    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')

    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-3.5-turbo-16k'):

    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    # Вывод количества токенов отключен
    # print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк\n\n')
    answer = completion.choices[0].message.content

    return answer


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток

            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'

        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        print(f'{answer}')  # Выводим ответ

    return processed_text, unprocessed_text

In [ ]:
!ls /content/drive/MyDrive/data_structure/

In [ ]:
# @title Формируем текст доклада
file_path = "/content/drive/MyDrive/data_structure/The AI revolution - Google's developers on the future of artificial intelligence _ 60 Minutes-880TBXMuzmk.txt"

# Вызываем функцию обработки для этого файла
processed_text, unprocessed_text = process_one_file(file_path, system, user)
